In [ ]:
base_path = "/content/drive/My Drive/Github/casbl-for-jadce-in-mtc"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append(f'{base_path}/frameworks')

from casbl import casbl

data = np.load(f'{base_path}/general_model/data/general_model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocations']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load(f'{base_path}/communication_model/data/communication_simulation_theta.npz', allow_pickle=True)
all_theta = data["all_theta"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_gamma.npz', allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_z.npz', allow_pickle=True)
all_z = data["all_z"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_noise.npz', allow_pickle=True)
all_noise = data["all_noise"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_received_signal.npz', allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load(f'{base_path}/communication_model/data/communication_simulation_general.npz', allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

pilot_length_index = 1
pilot_length=pilot_length_array[pilot_length_index]

snr_db_index = 2
snr_db=snr_db_array[snr_db_index]

alpha_array = [0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]
beta = 0.175

max_iter = 500

Mounted at /content/drive


In [ ]:
sample_range = range(0, 100)
num_samples = len(sample_range)

In [ ]:
# Final gamma and z results
gamma_result_all = np.empty((num_samples, len(alpha_array)), dtype=object)
z_result_all = np.empty((num_samples, len(alpha_array)), dtype=object)

In [ ]:
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    gamma = all_gamma[sample_index]  # Ground truth gamma values (true support)
    loc = all_DeviceLocations[sample_index]
    z = all_z[sample_index]

    for alpha_index in tqdm(range(len(alpha_array)), desc="Processing alpha Levels", leave=False, position=1):
        alpha = alpha_array[alpha_index]

        theta = all_theta[sample_index, pilot_length_index, :pilot_length]
        received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]

        # Compute noise
        snr = 10 ** (snr_db / 10)
        signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
        noise_power = signal_power / snr

        # Run CASBL
        gamma_result, z_result, gamma_history, z_history, iteration_count = casbl(
            theta, received_signal, noise_power, loc, alpha=alpha, beta=beta, max_iter=max_iter
        )

        # Save outputs
        gamma_result_all[sample_index-sample_range[0], alpha_index] = gamma_result.copy()
        z_result_all[sample_index-sample_range[0], alpha_index] = z_result.copy()

Processing alpha Levels:  78%|███████▊  | 7/9 [01:55<00:32, 16.11s/it]

Converged after 484 iterations



Processing alpha Levels:  89%|████████▉ | 8/9 [02:11<00:16, 16.22s/it]

Converged after 487 iterations



Processing Samples:   1%|          | 1/100 [02:28<4:04:12, 148.01s/it]

...

Converged after 458 iterations



Processing alpha Levels:  89%|████████▉ | 8/9 [02:14<00:16, 16.57s/it]

Converged after 409 iterations



Processing Samples: 100%|██████████| 100/100 [4:16:15<00:00, 153.76s/it]

Converged after 391 iterations


In [ ]:
np.savez_compressed(
    f'{base_path}/hyper_parameter_tuning/casbl/data/casbl_alpha_all_beta_0_175_snr_db_12_pilot_length_25.npz',
    gamma_result_all=gamma_result_all,
    z_result_all=z_result_all,
)

print("All results have been saved")

All results have been saved
